# Emulation of air quality in China

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lukeconibear/emulator_training/blob/main/emulator_training.ipynb)

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

![emulator.png](https://user-images.githubusercontent.com/19871268/128219782-a93b83a1-eafa-4e06-851e-92b4e10c256b.png)

Images [1](https://www.nasa.gov/content/a-portrait-of-global-winds/), [2](https://www.aidanscannell.com/post/gaussian-process-regression/), and [3](http://6degreesoffreedom.co/circle-random-sampling/).  

## Approach

### Problem
1. Identify
    - What is the problem you're trying to solve?
2. Inputs
    - What inputs go into the emulator?
3. Outputs
    - What will the emulator predict?

### Simulations
4. Design
    - Determine what the simulations will be (taking account of the inputs above).  
5. Run
    - Perform these simulations (including a control run).  
6. Evaluate
    - Compare the control simulation to measurements to ensure accurately represents the outputs above.

### Emulators
7. Design
    - Optimise the design of the emulators.  
8. Training
    - Train the emulators on the simulation data.
9. Evaluate
    - Test the emulators on the held-out test data.
10. Predict
    - Use the emulators for new predictions.

## Case study

### 1. Problem: Identify

Predict air quality from emission changes.  

### 2. Problem: Inputs

5 inputs, key anthropogenic emission sectors:  
- Residential (RES)
- Industrial (IND)
- Land transport (TRA)
- Agriculture (AGR)
- Power generation (ENE)

In [ ]:
sectors = ['RES', 'IND', 'TRA', 'AGR', 'ENE']

### 3. Problem: Outputs

1 output: PM$_{2.5}$ concentrations.  

In [ ]:
output = 'PM2_5_DRY'

### 4. Simulations: Design

Maximin [Latin-hypercube design](https://en.wikipedia.org/wiki/Latin_hypercube_sampling) using [pyDOE's](https://pythonhosted.org/pyDOE/index.html) [`lhs`](https://pythonhosted.org/pyDOE/randomized.html#latin-hypercube-lhs) method.
- For training, the number of simulation runs required is 10 times the number of inputs to adequately cover the parameter space (e.g., for 5 inputs, you'll need 50 runs).  
- For testing, have a separate set of simulations. A smaller number will suffice (e.g., for 5 inputs, 5 test simulations).  
- The Latin-hypercube design enables a near-random sample of values across the whole parameter space.  
- Maxi-min: Decides on the maximum and minimum values for the inputs, defaults to 1.0 and 0.0.  
- Choose number of iterations for the sampling (100,000 is common, though using 1,000 here is the example to run faster).  

In [ ]:
if IN_COLAB:
    !pip install --upgrade pyDOE

In [ ]:
import pandas as pd
from pyDOE import lhs
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
number_inputs = 5
iterations = 1_000 # use 100_000 in actual work

# create latin hypercube design, sampled between 0 (min) and 1 (max)
design_train = lhs(
    number_inputs, 
    samples=number_inputs * 10, 
    criterion='maximin', 
    iterations=iterations)

design_test = lhs(
    number_inputs, 
    samples=number_inputs, 
    criterion='maximin', 
    iterations=iterations)

# change scaling of max/min if appropriate
max_value = 1.5 # scale inputs up to 150% of baseline

design_train = design_train * max_value
design_test = design_test * max_value

In [ ]:
# convert numpy arrays to pandas dataframes
inputs_train = pd.DataFrame(data=design_train, columns=sectors)
inputs_test = pd.DataFrame(data=design_test, columns=sectors)

In [ ]:
inputs_train.describe()

In [ ]:
# plot the pairs as a histogram to check random sample over parameter space
sns.pairplot(inputs_train, diag_kind='hist')
plt.show()

In [ ]:
if IN_COLAB:
    !wget https://raw.githubusercontent.com/lukeconibear/emulator_training/main/latin_hypercube_inputs_train.csv
    !wget https://raw.githubusercontent.com/lukeconibear/emulator_training/main/latin_hypercube_inputs_test.csv

In [ ]:
# read in actual ones used
inputs_train = pd.read_csv('latin_hypercube_inputs_train.csv')
inputs_test = pd.read_csv('latin_hypercube_inputs_test.csv')

In [ ]:
# convert to arrays
X_train, X_test = inputs_train.values, inputs_test.values

### 5. Simulations: Run

Use a complex air quality model ([WRFChem](https://wrfchem-leeds.github.io/WRFotron/)).  
- 50 years of training simulations.  
- 5 years of testing simulations.  
- 1 year for the control simulation.  

*Warning: this takes a long time, even on a [high-performance computer](https://arcdocs.leeds.ac.uk/welcome.html).*

In [ ]:
if IN_COLAB:
    !wget https://raw.githubusercontent.com/lukeconibear/emulator_training/main/df_train.csv
    !wget https://raw.githubusercontent.com/lukeconibear/emulator_training/main/df_test.csv

In [ ]:
# bringing over data from my WRFChem runs 
df_train = pd.read_csv('df_train.csv', index_col='Unnamed: 0')
df_test = pd.read_csv('df_test.csv', index_col='Unnamed: 0')

In [ ]:
df_train.describe()

In [ ]:
gridcells = df_train[['lat', 'lon']].drop_duplicates().values.tolist()

print(f'There are {len(gridcells)} grid cells in the domain.')

In [ ]:
# pick a random grid cell (no reproducible seed, lucky dip!)
import random

random_gridcell_lat, random_gridcell_lon = random.choice(gridcells)

In [ ]:
# lets see what you picked ...
print(f'Lat = {random_gridcell_lat}')
print(f'Lon = {random_gridcell_lon}')

In [ ]:
# and what was the output for this random grid cell over all training simulations

random_gridcell_y_train = df_train.loc[df_train.lat == random_gridcell_lat].loc[df_train.lon == random_gridcell_lon][[output]].values
random_gridcell_y_test = df_test.loc[df_test.lat == random_gridcell_lat].loc[df_test.lon == random_gridcell_lon][[output]].values

random_gridcell_y_train

### 6. Simulations: Evaluate

Evaluate it against measurements to ensure it accurately predicts outputs.  

### 7. Emulators: Design

We optimised the design and hyperparameters of the model using genetic programming (automatic machine learning tool, [TPOT](http://epistasislab.github.io/tpot/)).  

Our final design included:
- Gaussian process regressors are implemented in [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html?highlight=gaussian%20process#sklearn.gaussian_process.GaussianProcessRegressor).  
- Preprocessing the inputs with a power transform ([Yeo-Johnson](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html)) to make them more Gaussian-like.  
- We used a [Matern](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.Matern.html?highlight=matern#sklearn.gaussian_process.kernels.Matern) kernel.  

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

In [ ]:
emulator = make_pipeline(
    PowerTransformer(),
    GaussianProcessRegressor(
        kernel=Matern(length_scale=3.4, nu=2.5), 
        n_restarts_optimizer=240, 
        normalize_y=False)
)
emulator

### 8. Emulators: Training

The emulators were trained on the 50 years of simulator data (in parallel on a high-performance computer).  
Example below for the single random grid cell.

In [ ]:
emulator.fit(X_train, random_gridcell_y_train)

### 9. Emulators: Evaluate

Predict unseen test data and evaluate against simulated values.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
def root_mean_squared_error(y_true, y_pred):
    return np.round(np.sqrt(np.abs(mean_squared_error(y_true, y_pred))), decimals=4)

def coefficient_of_determination(y_true, y_pred):
    return np.round(r2_score(y_true, y_pred), decimals=4)

In [ ]:
def create_scatter_plot(lat, lon, output, y_test, y_pred, rmse_test, r2_test):
    # setup the plot
    fig = plt.figure(1, figsize=(5, 5))
    gs = gridspec.GridSpec(1, 1)
    ax = fig.add_subplot(gs[0])
    
    # make it look nice
    ax.set_facecolor('whitesmoke')
    plt.title(f'Random grid cell\n(lat = {lat}, lon = {lon})', fontsize=14)
    limit = int(np.ceil(np.nanmax(random_gridcell_y_pred) / 5)) * 5
    plt.xlim([0, limit])
    plt.ylim([0, limit])
    ticks = np.linspace(0, 20, limit // int(limit / 5))
    plt.xticks(ticks, fontsize=14)
    plt.yticks(ticks, fontsize=14)
    label = 'annual-mean PM$_{2.5}$\nconcentrations (${\mu}g$ $m^{-3}$)'
    plt.xlabel('Simulator, ' + label, fontsize=14)
    plt.ylabel('Emulator, ' + label, fontsize=14)
    x = np.arange(2 * np.ceil(limit))
    plt.plot(x, x, color='grey', linestyle='--')
    plt.plot(x, 0.5 * x, color='grey', linestyle='--')
    plt.plot(x, 2 * x, color='grey', linestyle='--')
    text = "R$^2$ = " + str(r2_test) + "\nRMSE = " + str(rmse_test)
    at = matplotlib.offsetbox.AnchoredText(text, prop=dict(size=14), frameon=True, loc='upper left')
    at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
    ax.add_artist(at)
    
    # show the data
    plt.scatter(np.vstack(y_test), np.vstack(y_pred), s=50)

In [ ]:
random_gridcell_y_pred = emulator.predict(X_test)
random_gridcell_y_pred

In [ ]:
random_gridcell_rmse_test = root_mean_squared_error(random_gridcell_y_test, random_gridcell_y_pred)
random_gridcell_r2_test = coefficient_of_determination(random_gridcell_y_test, random_gridcell_y_pred)

print(f'The random grid cell has a RSME value of: {random_gridcell_rmse_test} ugm-3')
print(f'The random grid cell has a R^2 value of:  {random_gridcell_r2_test}')

In [ ]:
# example for the random grid cell
create_scatter_plot(
    random_gridcell_lat,
    random_gridcell_lon,
    output, 
    random_gridcell_y_test, 
    random_gridcell_y_pred, 
    random_gridcell_rmse_test,
    random_gridcell_r2_test)

We developed 1 emulator per grid cell from the simulator (15,278 grid cells) to capture the spatial distribution of the output.

### 10. Emulators: Predict

Create your own custom emission configuration and use your individual emulator to predict what the PM$_{2.5}$ concentrations would be. Remember, this is still just for a single grid cell.

In [ ]:
# select values between 0.0 and 1.5
fraction_of_residential_emissions      = 0.6
fraction_of_industrial_emissions       = 1.4
fraction_of_land_transport_emissions   = 1.3
fraction_of_agricultural_emissions     = 1.2
fraction_of_power_generation_emissions = 0.9

In [ ]:
# placing these into a stacked column array for the emulator
custom_emission_configuration = np.array([[
    fraction_of_residential_emissions,
    fraction_of_industrial_emissions,
    fraction_of_land_transport_emissions,
    fraction_of_agricultural_emissions,
    fraction_of_power_generation_emissions]])

In [ ]:
# now predict the output for these inputs
prediction = emulator.predict(custom_emission_configuration)

In [ ]:
# this is just some setup for the table and plot below
from tabulate import tabulate

table = tabulate(
    [
        ['Residential', int((fraction_of_residential_emissions * 100) - 100)],
        ['Industrial', int((fraction_of_industrial_emissions * 100) - 100)],
        ['Land transport', int((fraction_of_land_transport_emissions * 100) - 100)],
        ['Agricultural', int((fraction_of_agricultural_emissions * 100) - 100)],
        ['Power generation', int((fraction_of_power_generation_emissions * 100) - 100)],
    ],
    headers=['Emission sector', 'Fractional reduction (%)'])

# ---
import folium

background_map = folium.Map(
    location=[random_gridcell_lat, random_gridcell_lon], 
    zoom_start=5, 
    tiles="Stamen Terrain")

tooltip = "Click me!"

folium.Marker(
    [random_gridcell_lat, random_gridcell_lon],
    popup=f"<b>Emulator</b> {float(prediction):.1f} \u03BCg/m\u00b3",
    tooltip=tooltip
).add_to(background_map);

In [ ]:
print(f'You choose a custom emission configuration where:')
print()
print(table)
print()
print(f'The predicted PM2.5 concentration would be {float(prediction):.1f}' + ' \u03BCg/m\u00b3')
print()
background_map

All 15,278 emulators were then used to predict air quality concentrations for all emission configurations within a 0−150% matrix of emission scaling factors at 20% increments (32,768 emission configurations).  

For example, one emission configuration could be (*emulator inputs*):  

| Sector           | Fractional emissions              |
| ---              | ---                               |
| Residential      | 0.6 (-40% from the 2015 baseline) | 
| Industrial       | 0.4 (-60% from the 2015 baseline) | 
| Land transport   | 0.8 (-20% from the 2015 baseline) | 
| Agricultural     | 0.8 (-20% from the 2015 baseline) | 
| Power generation | 0.8 (-20% from the 2015 baseline) | 

This result of this emission configuration should be (*emulator outputs*):
- PM$_{2.5}$ exposure in China of 34.3 ${\mu}g$ $m^{-3}$ (-28%).  
    - Reaching the National Air Quality Target (NAQT) of 35 ${\mu}g$ $m^{-3}$.  
- 361,900 avoided premature deaths per year (-17%).
    - 1,781,800 remaining premature deaths per year.

Play around with the interactive plot to see what air quality and public health benefits you can find.  

*The code for the interactive plot is [here](https://github.com/lukeconibear/emulator_training/blob/main/interactive_plot.py).* 

**Questions?**  
- Removing emissions from which 1 sector would attain the NAQT?
- Removing emissions from which 2 sectors would attins the World Health Organization (WHO) Interim Target 2 (25 ${\mu}g$ $m^{-3}$)?
- Does removing emissions from all of these 5 sectors attain the WHO Annual Guideline (5 ${\mu}g$ $m^{-3}$)?
    - Why?



In [ ]:
from IPython.display import IFrame

IFrame("https://emulator.pythonanywhere.com/", width=800, height=800)

## Further information
Short-term air quality prediction.
- [Code](https://github.com/lukeconibear/emulator).
- Papers:
    - *Conibear, L. Reddington, C. L., Silver, B. J., Chen, Y., Knote, C., Arnold, S. R., Spracklen, D. V. (2021). Statistical emulation of winter ambient fine particulate matter concentrations from emission changes in China, GeoHealth, [DOI](https://doi.org/10.1029/2021GH000391).*

Long-term air quality and public health prediction
- [Code](https://github.com/lukeconibear/emulator_annual).
- Papers:
    - *Conibear, L. Reddington, C. L., Silver, B. J., Chen, Y., Knote, C., Arnold, S. R., Spracklen, D. V. (2021, in prep.). Machine Learning Emulation of Long−Term Air Pollution Exposure and Disease Burden from Emission Changes in China.*
    - *Conibear, L. Reddington, C. L., Silver, B. J., Arnold, S. R., Turnock, S. T., Klimont, Z., Spracklen, D. V. (2021, in prep.). The Contribution of Emission Sources to the Future Air Pollution Disease Burden in China.*
    - *Conibear, L. Reddington, C. L., Silver, B. J., Chen, Y., Knote, C., Arnold, S. R., Spracklen, D. V. (2021, in prep.). Emission Sector Impacts on Air Quality and Public Health in China from 2010−2020.*
    
Health impact assessment used the Global Exposure Mortality Model (GEMM) from [Burnett et al., (2018)](https://doi.org/10.1073/pnas.1803222115). See, our papers for more detail on this method, and the code [here](https://github.com/lukeconibear/health_impact_assessment).